In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC CLASSIFIER USING MAMMOGRAM PATCHES IN BCDR
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time


vid='v003_4'

In [2]:
# Create CNN

IMG_SIZE=224

from tensorflow.keras.applications.xception import Xception
base_model=Xception(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(1,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [4]:
len(model.layers)

139

In [5]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy']) 

In [6]:
# Image preprocessing and data augmentation

batch_size=8

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('mass_ben_mal_class02/train0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 530 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [7]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [8]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mass_ben_mal_class_xception_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16852\1152529500.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/3
66/66 [==============================] - 58s 807ms/step - loss: 0.5830 - accuracy: 0.7241 - val_loss: 0.5254 - val_accuracy: 0.7721 - lr: 0.0010
Epoch 2/3
66/66 [==============================] - 51s 777ms/step - loss: 0.4835 - accuracy: 0.7778 - val_loss: 0.5465 - val_accuracy: 0.7794 - lr: 0.0010
Epoch 3/3
66/66 [==============================] - 54s 820ms/step - loss: 0.4230 - accuracy: 0.8295 - val_loss: 0.4498 - val_accuracy: 0.7574 - lr: 0.0010


In [9]:
#Training stage 2
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16852\81111685.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/10
66/66 [==============================] - 104s 1s/step - loss: 0.4562 - accuracy: 0.7931 - val_loss: 0.5960 - val_accuracy: 0.8162 - lr: 1.0000e-04
Epoch 2/10
66/66 [==============================] - 92s 1s/step - loss: 0.3599 - accuracy: 0.8333 - val_loss: 0.5245 - val_accuracy: 0.8015 - lr: 1.0000e-04
Epoch 3/10
66/66 [==============================] - 93s 1s/step - loss: 0.4004 - accuracy: 0.8333 - val_loss: 0.4321 - val_accuracy: 0.8015 - lr: 1.0000e-04
Epoch 4/10
66/66 [==============================] - 93s 1s/step - loss: 0.3643 - accuracy: 0.8525 - val_loss: 0.3929 - val_accuracy: 0.8382 - lr: 1.0000e-04
Epoch 5/10
66/66 [==============================] - 92s 1s/step - loss: 0.3522 - accuracy: 0.8640 - val_loss: 0.3727 - val_accuracy: 0.8603 - lr: 1.0000e-04
Epoch 6/10
66/66 [==============================] - 91s 1s/step - loss: 0.3407 - accuracy: 0.8525 - val_loss: 0.4263 - val_accuracy: 0.8235 - lr: 1.0000e-04
Epoch 7/10
66/66 [==============================] - 91s 1

In [10]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16852\2279829289.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


66/66 [==============================] - 226s 3s/step - loss: 0.3476 - accuracy: 0.8716 - val_loss: 0.4098 - val_accuracy: 0.8529 - lr: 1.0000e-05
Epoch 2/150
66/66 [==============================] - 208s 3s/step - loss: 0.3252 - accuracy: 0.8716 - val_loss: 0.4029 - val_accuracy: 0.8235 - lr: 1.0000e-05
Epoch 3/150
66/66 [==============================] - 208s 3s/step - loss: 0.2985 - accuracy: 0.8870 - val_loss: 0.3807 - val_accuracy: 0.8088 - lr: 1.0000e-05
Epoch 4/150
66/66 [==============================] - 209s 3s/step - loss: 0.3255 - accuracy: 0.8736 - val_loss: 0.3572 - val_accuracy: 0.8382 - lr: 1.0000e-05
Epoch 5/150
66/66 [==============================] - 207s 3s/step - loss: 0.2737 - accuracy: 0.8870 - val_loss: 0.3588 - val_accuracy: 0.8456 - lr: 1.0000e-05
Epoch 6/150
66/66 [==============================] - 208s 3s/step - loss: 0.3092 - accuracy: 0.8544 - val_loss: 0.3556 - val_accuracy: 0.8382 - lr: 1.0000e-05
Epoch 7/150
66/66 [==============================] - 204s 

In [11]:
model.metrics_names

['loss', 'accuracy']

In [12]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('mass_ben_mal_class02/valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(valid_generator.classes, Y_pred)
prec=precision_score(valid_generator.classes, y_pred,pos_label=1)
rec=recall_score(valid_generator.classes, y_pred,pos_label=1)
spe=recall_score(valid_generator.classes, y_pred,pos_label=0)
f1s=f1_score(valid_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(valid_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 136 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16852\3759042783.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(valid_generator)


Confusion Matrix:
[[86  6]
 [13 31]]
Sensitivity:  0.7045454545454546
Specificity:  0.9347826086956522
Precision score: 0.8378378378378378
Accuracy: 0.8602941176470589
F1 score: 0.7654320987654323
MCC score: 0.6720871016872976
ROC AUC score: 0.906620553359684


In [13]:
#Print estimation result for ensemble model
print(Y_pred)
print(valid_generator.classes)

[[1.03177592e-01]
 [2.62754828e-01]
 [1.55333122e-02]
 [1.71483845e-01]
 [4.28988874e-01]
 [1.17196470e-01]
 [1.95231274e-01]
 [1.76836446e-01]
 [1.79211900e-01]
 [1.94086224e-01]
 [2.70342920e-03]
 [7.10162101e-03]
 [1.82112426e-01]
 [2.41236106e-01]
 [1.21807843e-01]
 [2.56159641e-02]
 [3.55172087e-03]
 [2.13626653e-01]
 [9.63755026e-02]
 [3.77199128e-02]
 [6.00253224e-01]
 [5.94060794e-02]
 [3.03015187e-02]
 [7.31700212e-02]
 [2.57631779e-01]
 [4.89380896e-01]
 [7.52385557e-02]
 [7.01094791e-02]
 [7.90372267e-02]
 [2.22336277e-01]
 [9.59847472e-04]
 [2.69195791e-02]
 [1.62164524e-01]
 [6.70284554e-02]
 [1.53305009e-01]
 [6.36513308e-02]
 [1.92783400e-01]
 [1.09390497e-01]
 [7.37962127e-02]
 [9.68032256e-02]
 [1.27008766e-01]
 [1.71419173e-01]
 [1.69381443e-02]
 [1.71310874e-03]
 [4.57458086e-02]
 [1.16518088e-01]
 [3.06663662e-01]
 [7.39154518e-01]
 [2.45208535e-02]
 [3.30138393e-02]
 [2.17943504e-01]
 [1.29865050e-01]
 [2.10810150e-03]
 [2.82495674e-02]
 [2.06012741e-01]
 [4.300574

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('mass_ben_mal_class02/test', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
spe=recall_score(test_generator.classes, y_pred,pos_label=0)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

print('Sensitivity: ', rec)
print('Specificity: ', spe)
print('Precision score:', prec)
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('F1 score:', f1s)
print('MCC score:', mcc)
print('ROC AUC score:', ras)

Found 169 images belonging to 2 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_16852\3439002409.py:15: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator)


Confusion Matrix:
[[102  11]
 [ 21  35]]
Sensitivity:  0.625
Specificity:  0.9026548672566371
Precision score: 0.7608695652173914
Accuracy: 0.8106508875739645
F1 score: 0.6862745098039216
MCC score: 0.5580225023003181
ROC AUC score: 0.8596713021491783


In [15]:
#Print estimation result for ensemble model
print(Y_pred)
print(test_generator.classes)

[[0.45172766]
 [0.26087222]
 [0.14144732]
 [0.1015777 ]
 [0.20474544]
 [0.00505683]
 [0.5690298 ]
 [0.00871263]
 [0.05757531]
 [0.04624037]
 [0.02274937]
 [0.15244003]
 [0.14509782]
 [0.3999947 ]
 [0.0610083 ]
 [0.02977579]
 [0.5576038 ]
 [0.3391983 ]
 [0.00338048]
 [0.3064339 ]
 [0.29231966]
 [0.08302265]
 [0.09386899]
 [0.11127286]
 [0.03284694]
 [0.06243689]
 [0.02938769]
 [0.05704198]
 [0.6002592 ]
 [0.3448021 ]
 [0.04814851]
 [0.12902051]
 [0.16660789]
 [0.16269183]
 [0.08752027]
 [0.00321364]
 [0.05644094]
 [0.48161158]
 [0.01990118]
 [0.64663357]
 [0.22311792]
 [0.29992536]
 [0.01525725]
 [0.0528871 ]
 [0.09156395]
 [0.0468106 ]
 [0.04727623]
 [0.01898573]
 [0.7270687 ]
 [0.01792516]
 [0.0401619 ]
 [0.34972662]
 [0.1645159 ]
 [0.00699084]
 [0.08860946]
 [0.27224597]
 [0.17928837]
 [0.18965589]
 [0.40767676]
 [0.02627094]
 [0.12696816]
 [0.03945727]
 [0.06457432]
 [0.58048224]
 [0.14433712]
 [0.15697779]
 [0.15794423]
 [0.10979183]
 [0.06679354]
 [0.01034696]
 [0.25132295]
 [0.02